In [1]:
import pandas as pd
import numpy as np
from os.path import join

RESULTS_PATH = r"C:\Users\shami\OneDrive\Desktop\Master_Thesis\ADA_GWU_Master_Thesis\Reports\Experiment_Results"

In [2]:
raw_results = pd.read_excel(join(RESULTS_PATH,"ALL_EXPERIMENT_RESULTS.xlsx"))

In [3]:
raw_results.replace({"DDBoost":"DCBoost"}, inplace=True)

In [4]:
raw_results = raw_results[raw_results.Dataset != 'PTP_Data'].copy()

In [5]:
raw_results["Overall_Score"] = np.dot(raw_results[['Accuracy', "F1 Score", 'AUC Score']].values,
                                      np.array([0.05, 0.475, 0.475])
                              )

In [6]:
raw_results["Overall_Score_R"] = np.dot(raw_results[['Accuracy', 'Recall', 'Precision', 'AUC Score']].values,
                                        np.array([0.05, 0.55, 0.2, 0.2]))

In [8]:
raw_results.to_excel(join(RESULTS_PATH,"R1.xlsx"),index=False)

In [20]:
raw_results

,Model,Sample,Accuracy,F1 Score,Precision,Recall,AUC Score,Training Time (s),Dataset,Overall_Score,Overall_Score_R
0,SelfBoostingXGB,Train,0.868228,0.689817,0.797656,0.607664,0.925674,6.953560,Adult,0.810770,0.722293
1,SelfBoostingXGB,Test,0.861627,0.675689,0.803394,0.583015,0.919570,6.953560,Adult,0.800829,0.708332
2,PSM,Train,0.866311,0.702737,0.757519,0.655345,0.924842,43.545517,Adult,0.816416,0.740228
3,PSM,Test,0.857323,0.686216,0.752024,0.630998,0.917987,43.545517,Adult,0.804863,0.723917
4,SSM,Train,0.866416,0.700100,0.763202,0.646636,0.923849,70.820222,Adult,0.814697,0.736381
...,...,...,...,...,...,...,...,...,...,...,...
1199,LOF_IF_RandomForest,Test,0.813667,0.400857,0.682482,0.283763,0.769984,35.391081,Taiwan_Credit_Scoring,0.596833,0.487246
1200,LOF_IF_XGB,Train,0.866085,0.572831,0.770925,0.455729,0.905400,1.120415,Wine,0.745464,0.629220
1201,LOF_IF_XGB,Test,0.833846,0.465347,0.610390,0.376000,0.871299,1.120415,Wine,0.676599,0.544830
1202,LOF_IF_RandomForest,Train,0.822644,0.224383,0.810811,0.130208,0.850696,1.511070,Wine,0.551794,0.445048


In [32]:
pivotted = raw_results.pivot_table(index=["Model", "Dataset","Training Time (s)"], 
                              columns="Sample", 
                              values=["Accuracy", "F1 Score", "Precision", "Recall", "AUC Score", 
                                      "Overall_Score", "Overall_Score_R"], 
                              aggfunc='first')
pivotted.columns = ['{}_{}'.format(col[1], col[0]) for col in pivotted.columns]
pivotted.reset_index(inplace=True)

In [34]:
pivotted.Model.unique()

array(['ADASYN_Logistic', 'ADASYN_RandomForest', 'ADASYN_XGB',
       'BalancedBagging', 'DCBoost', 'EasyEnsemble', 'IF_Logistic',
       'IF_RandomForest', 'IF_XGB', 'IsolationForest',
       'LOF_IF_RandomForest', 'LOF_IF_XGB', 'LOF_Logistic',
       'LOF_RandomForest', 'LOF_XGB', 'Logistic', 'Logistic_ClassWeights',
       'PSM', 'RAMOBoost', 'ROS_Logistic', 'ROS_RandomForest', 'ROS_XGB',
       'RUSBoost', 'RUS_Logistic', 'RUS_RandomForest', 'RUS_XGB',
       'RandomFores_ClassWeights', 'RandomForest', 'SMOTEBagging',
       'SMOTEBoost', 'SMOTETomek_Logistic', 'SMOTETomek_RandomForest',
       'SMOTETomek_XGB', 'SMOTE_Logistic', 'SMOTE_RandomForest',
       'SMOTE_XGB', 'SSM', 'SelfBoostingXGB', 'Tomek_Logistic',
       'Tomek_RandomForest', 'Tomek_XGB', 'XGB', 'XGB_ClassWeights'],
      dtype=object)

In [109]:
baseline_models = pivotted[pivotted.Model.isin(['XGB','Logistic','RandomForest'])].copy()

In [119]:
baseline_models["Easy_Nature"] = np.where(baseline_models["Test_Overall_Score"]>0.90,1,0)

In [120]:
easy_nature_df = pd.DataFrame(baseline_models.groupby(by="Dataset").Easy_Nature.max()).reset_index()

In [121]:
easy_nature_df

,Dataset,Easy_Nature
0,Adult,0
1,Churn_Ecom,0
2,CoverType,0
3,GMC_Credit_Scoring,0
4,Nursery,1
5,PTB_Online,0
6,Taiwan_Credit_Scoring,0
7,Wine,0
8,cc_fraud_05,1
9,cc_fraud_1,1


In [54]:
s2 = pivotted.merge(easy_nature_df, how='left', on='Dataset')

In [75]:
s2["OverFitting_Flag"] = np.where(s2["Train_Overall_Score"] >= s2["Test_Overall_Score"]*1.10 ,1,0)

s2["OverFitting_Index"] = s2["Train_Overall_Score"]/s2["Test_Overall_Score"]

In [76]:
s2[s2.OverFitting_Flag==1]

,Model,Dataset,Training Time (s),Test_AUC Score,Train_AUC Score,Test_Accuracy,Train_Accuracy,Test_F1 Score,Train_F1 Score,Test_Overall_Score,Train_Overall_Score,Test_Overall_Score_R,Train_Overall_Score_R,Test_Precision,Train_Precision,Test_Recall,Train_Recall,Easy_Nature,OverFitting_Flag,OverFitting_Index
61,DCBoost,cc_fraud_1,3.140116,0.970167,0.999933,0.996646,0.999695,0.821622,0.984576,0.900932,0.992626,0.845109,0.984095,0.873563,0.997396,0.775510,0.972081,1,1,1.101777
96,IF_RandomForest,PTB_Online,1.401543,0.899024,0.922787,0.850770,0.874290,0.226891,0.327711,0.577348,0.637701,0.449782,0.521733,0.771429,0.918919,0.133005,0.199413,0,1,1.104535
135,LOF_IF_RandomForest,PTB_Online,3.105344,0.896537,0.920364,0.858070,0.884654,0.273859,0.417122,0.598842,0.679539,0.485304,0.562605,0.868421,0.932790,0.162562,0.268622,0,1,1.134755
150,LOF_IF_XGB,Wine,1.120415,0.871299,0.905400,0.833846,0.866085,0.465347,0.572831,0.676599,0.745464,0.544830,0.629220,0.610390,0.770925,0.376000,0.455729,0,1,1.101781
174,LOF_RandomForest,PTB_Online,1.250936,0.899063,0.922604,0.853204,0.883933,0.242678,0.410796,0.584987,0.677562,0.462155,0.560251,0.805556,0.933472,0.142857,0.263343,0,1,1.158251
230,PSM,cc_fraud_1,49.967399,0.941230,0.999976,0.997053,0.999695,0.839779,0.984733,0.895832,0.992721,0.847762,0.987657,0.915663,0.987245,0.775510,0.982234,1,1,1.108156
231,PSM,cc_fraud_2,34.536986,0.933571,0.999965,0.993089,0.999368,0.800000,0.984375,0.873101,0.992530,0.833157,0.987477,0.809524,0.986577,0.790698,0.982183,1,1,1.136787
475,SSM,Wine,1.913174,0.871657,0.926765,0.833846,0.881478,0.517857,0.649469,0.701712,0.792785,0.588395,0.691574,0.585859,0.778182,0.464000,0.557292,0,1,1.129787
477,SSM,cc_fraud_1,48.962227,0.903406,0.999990,0.996443,0.999644,0.810811,0.982097,0.864075,0.991473,0.823835,0.983959,0.862069,0.989691,0.765306,0.974619,1,1,1.147439
478,SSM,cc_fraud_2,37.455514,0.942674,0.999984,0.993902,0.999368,0.819277,0.984410,0.886622,0.992555,0.843114,0.988273,0.850000,0.984410,0.790698,0.984410,1,1,1.119480


In [81]:
s2.groupby(by="Model").OverFitting_Index.mean().sort_values()

Model
IsolationForest             0.991788
SMOTETomek_Logistic         1.007565
ROS_Logistic                1.007960
RUS_Logistic                1.008548
Logistic_ClassWeights       1.011050
ADASYN_Logistic             1.011466
SMOTE_Logistic              1.011976
IF_Logistic                 1.014935
LOF_Logistic                1.015501
SMOTEBagging                1.016389
ROS_RandomForest            1.016408
Logistic                    1.016724
BalancedBagging             1.017734
SMOTETomek_RandomForest     1.018011
RUS_RandomForest            1.018258
EasyEnsemble                1.018282
Tomek_Logistic              1.018754
RandomFores_ClassWeights    1.019237
ADASYN_RandomForest         1.019519
SMOTE_RandomForest          1.020611
RandomForest                1.020855
IF_RandomForest             1.021080
Tomek_RandomForest          1.021404
SMOTEBoost                  1.021661
LOF_IF_RandomForest         1.023402
RUSBoost                    1.024760
RAMOBoost                   1.02

In [85]:
s2.drop(columns=['Train_AUC Score',  'Train_Accuracy', 
                 'Train_F1 Score',  'Train_Overall_Score',
                 'Train_Overall_Score_R', 
                 'Train_Precision',  'Train_Recall'], inplace = True)

In [ ]:
#s2['rn_auc'] = 

In [96]:
s2 = s2.sort_values(by=['Dataset', 'Test_AUC Score', 'Test_Overall_Score'], ascending=[True, False, False]).copy()

# Now we'll use rank() to assign a row number within each 'Dataset'
s2['rank_auc'] = s2.groupby('Dataset').cumcount() + 1

In [97]:
s2 = s2.sort_values(by=['Dataset', 'Test_F1 Score', 'Test_Overall_Score'], ascending=[True, False, False]).copy()

# Now we'll use rank() to assign a row number within each 'Dataset'
s2['rank_f1'] = s2.groupby('Dataset').cumcount() + 1

In [98]:
s2 = s2.sort_values(by=['Dataset', 'Test_Precision', 'Test_Overall_Score'], ascending=[True, False, False]).copy()

# Now we'll use rank() to assign a row number within each 'Dataset'
s2['rank_precision'] = s2.groupby('Dataset').cumcount() + 1

In [99]:
s2 = s2.sort_values(by=['Dataset', 'Test_Recall', 'Test_Overall_Score'], ascending=[True, False, False]).copy()

# Now we'll use rank() to assign a row number within each 'Dataset'
s2['rank_recall'] = s2.groupby('Dataset').cumcount() + 1

In [100]:
s2 = s2.sort_values(by=['Dataset', 'Test_Overall_Score', 'Test_Overall_Score_R'], ascending=[True, False, False]).copy()

# Now we'll use rank() to assign a row number within each 'Dataset'
s2['rank_overall'] = s2.groupby('Dataset').cumcount() + 1

In [101]:
s2 = s2.sort_values(by=['Dataset', 'Test_Overall_Score_R', 'Test_Overall_Score'], ascending=[True, False, False]).copy()

# Now we'll use rank() to assign a row number within each 'Dataset'
s2['rank_overall_2'] = s2.groupby('Dataset').cumcount() + 1

In [103]:
s2[s2.Dataset == 'Adult'].sort_values(by="rank_f1")

,Model,Dataset,Training Time (s),Test_AUC Score,Test_Accuracy,Test_F1 Score,Test_Overall_Score,Test_Overall_Score_R,Test_Precision,Test_Recall,Easy_Nature,OverFitting_Flag,OverFitting_Index,rank_auc,rank_f1,rank_precision,rank_recall,rank_overall,rank_overall_2
52,DCBoost,Adult,4.009983,0.899710,0.850289,0.718959,0.811382,0.782611,0.670835,0.774522,0,0,1.007260,19,1,19,14,1,13
273,ROS_XGB,Adult,1.677647,0.917925,0.815643,0.698178,0.808431,0.815995,0.586486,0.862420,0,0,1.004900,9,2,25,7,3,1
546,XGB_ClassWeights,Adult,1.064009,0.918689,0.815118,0.697579,0.808483,0.815953,0.585640,0.862420,0,0,1.006694,5,3,26,6,2,2
325,RUS_XGB,Adult,0.642692,0.919726,0.814803,0.696386,0.808393,0.814254,0.585528,0.859023,0,0,1.008006,1,4,27,9,4,3
26,ADASYN_XGB,Adult,21.214929,0.902986,0.834541,0.690859,0.798803,0.761998,0.641998,0.747771,0,0,1.008051,18,5,22,16,15,15
520,Tomek_XGB,Adult,40.398075,0.917716,0.861522,0.689135,0.806330,0.722936,0.774364,0.620807,0,0,1.010686,10,6,11,24,5,21
286,RUSBoost,Adult,2.300796,0.912850,0.811234,0.688927,0.801406,0.804384,0.581314,0.845435,0,0,1.005247,12,7,28,12,8,4
455,SMOTE_XGB,Adult,5.490145,0.905138,0.835696,0.687562,0.798318,0.754744,0.648832,0.731210,0,0,1.010746,16,8,21,17,17,16
416,SMOTETomek_XGB,Adult,76.474304,0.905023,0.837270,0.686615,0.797892,0.750498,0.655345,0.721019,0,0,1.011348,17,9,20,18,18,17
221,PSM,Adult,43.545517,0.917987,0.857323,0.686216,0.804863,0.723917,0.752024,0.630998,0,0,1.014355,8,10,13,22,6,19


In [104]:
s2[s2.Model == 'DCBoost']

,Model,Dataset,Training Time (s),Test_AUC Score,Test_Accuracy,Test_F1 Score,Test_Overall_Score,Test_Overall_Score_R,Test_Precision,Test_Recall,Easy_Nature,OverFitting_Flag,OverFitting_Index,rank_auc,rank_f1,rank_precision,rank_recall,rank_overall,rank_overall_2
52,DCBoost,Adult,4.009983,0.899710,0.850289,0.718959,0.811382,0.782611,0.670835,0.774522,0,0,1.007260,19,1,19,14,1,13
53,DCBoost,Churn_Ecom,5.086403,0.751310,0.718902,0.334053,0.551493,0.586058,0.225389,0.645042,0,0,1.037845,1,2,29,4,1,3
54,DCBoost,CoverType,4.695514,0.987269,0.983400,0.774457,0.885990,0.837779,0.748031,0.802817,0,0,1.013655,13,1,14,23,1,5
55,DCBoost,GMC_Credit_Scoring,5.774926,0.823627,0.858556,0.392131,0.620413,0.641375,0.274062,0.688926,0,0,1.006046,32,1,23,17,1,17
56,DCBoost,Nursery,1.535473,0.999932,0.997685,0.957746,0.979782,0.973045,0.944444,0.971429,1,0,1.006634,5,3,9,23,3,3
57,DCBoost,PTB_Online,3.011782,0.913181,0.879157,0.671082,0.796483,0.760017,0.608000,0.748768,0,0,1.045311,15,2,20,15,2,13
58,DCBoost,Taiwan_Credit_Scoring,4.013297,0.782398,0.744667,0.522444,0.657033,0.632087,0.443386,0.635812,0,0,1.041179,12,8,30,6,6,4
59,DCBoost,Wine,2.762780,0.866781,0.807692,0.600639,0.737409,0.727341,0.500000,0.752000,0,0,1.076223,13,1,18,19,2,10
60,DCBoost,cc_fraud_05,5.067688,0.970089,0.998476,0.839858,0.909649,0.866899,0.861314,0.819444,1,0,1.089394,10,7,16,23,7,7
61,DCBoost,cc_fraud_1,3.140116,0.970167,0.996646,0.821622,0.900932,0.845109,0.873563,0.775510,1,1,1.101777,12,12,16,19,7,8


In [106]:
s2["rank_auc_score"] = 1/s2.rank_auc
s2["rank_f1_score"] = 1/s2.rank_f1
s2["rank_precision_score"] = 1/s2.rank_precision
s2["rank_recall_score"] = 1/s2.rank_recall
s2["rank_overall_score"] = 1/s2.rank_overall
s2["rank_overall_2_score"] = 1/s2.rank_overall_2

In [107]:
s2

,Model,Dataset,Training Time (s),Test_AUC Score,Test_Accuracy,Test_F1 Score,Test_Overall_Score,Test_Overall_Score_R,Test_Precision,Test_Recall,...,rank_precision,rank_recall,rank_overall,rank_overall_2,rank_auc_score,rank_f1_score,rank_precision_score,rank_recall_score,rank_overall_score,rank_overall_2_score
273,ROS_XGB,Adult,1.677647,0.917925,0.815643,0.698178,0.808431,0.815995,0.586486,0.862420,...,25,7,3,1,0.111111,0.500000,0.040000,0.142857,0.333333,1.000000
546,XGB_ClassWeights,Adult,1.064009,0.918689,0.815118,0.697579,0.808483,0.815953,0.585640,0.862420,...,26,6,2,2,0.200000,0.333333,0.038462,0.166667,0.500000,0.500000
325,RUS_XGB,Adult,0.642692,0.919726,0.814803,0.696386,0.808393,0.814254,0.585528,0.859023,...,27,9,4,3,1.000000,0.250000,0.037037,0.111111,0.250000,0.333333
286,RUSBoost,Adult,2.300796,0.912850,0.811234,0.688927,0.801406,0.804384,0.581314,0.845435,...,28,12,8,4,0.083333,0.142857,0.035714,0.083333,0.125000,0.250000
65,EasyEnsemble,Adult,13.624655,0.905740,0.799685,0.675621,0.791130,0.797861,0.563368,0.843737,...,29,13,19,5,0.066667,0.062500,0.034483,0.076923,0.052632,0.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,Logistic_ClassWeights,cc_fraud_4,0.251029,0.975220,0.942276,0.564417,0.778441,0.839186,0.403509,0.938776,...,42,7,42,39,0.034483,0.023810,0.023810,0.142857,0.023810,0.025641
259,ROS_Logistic,cc_fraud_4,0.395190,0.979505,0.948780,0.588235,0.792116,0.834981,0.432692,0.918367,...,40,11,40,40,0.035714,0.024390,0.025000,0.090909,0.025000,0.025000
402,SMOTETomek_Logistic,cc_fraud_4,1.316931,0.950094,0.960976,0.647059,0.806696,0.833095,0.505747,0.897959,...,37,33,39,41,0.026316,0.027027,0.027027,0.030303,0.025641,0.024390
311,RUS_Logistic,cc_fraud_4,0.124177,0.958354,0.941463,0.555556,0.766180,0.823492,0.398230,0.918367,...,43,12,43,42,0.027778,0.023256,0.023256,0.083333,0.023256,0.023810


In [108]:
s2.to_excel(join(RESULTS_PATH,"ANALYSIS_STEP1.xlsx"),index=False)

## COMPARISON OF BEST BASELINE AND BEST MODEL

In [122]:
baseline_models = s2[s2.Model.isin(['XGB','Logistic','RandomForest'])].copy()

In [167]:
s2

,Model,Dataset,Training Time (s),Test_AUC Score,Test_Accuracy,Test_F1 Score,Test_Overall_Score,Test_Overall_Score_R,Test_Precision,Test_Recall,...,rank_precision,rank_recall,rank_overall,rank_overall_2,rank_auc_score,rank_f1_score,rank_precision_score,rank_recall_score,rank_overall_score,rank_overall_2_score
273,ROS_XGB,Adult,1.677647,0.917925,0.815643,0.698178,0.808431,0.815995,0.586486,0.862420,...,25,7,3,1,0.111111,0.500000,0.040000,0.142857,0.333333,1.000000
546,XGB_ClassWeights,Adult,1.064009,0.918689,0.815118,0.697579,0.808483,0.815953,0.585640,0.862420,...,26,6,2,2,0.200000,0.333333,0.038462,0.166667,0.500000,0.500000
325,RUS_XGB,Adult,0.642692,0.919726,0.814803,0.696386,0.808393,0.814254,0.585528,0.859023,...,27,9,4,3,1.000000,0.250000,0.037037,0.111111,0.250000,0.333333
286,RUSBoost,Adult,2.300796,0.912850,0.811234,0.688927,0.801406,0.804384,0.581314,0.845435,...,28,12,8,4,0.083333,0.142857,0.035714,0.083333,0.125000,0.250000
65,EasyEnsemble,Adult,13.624655,0.905740,0.799685,0.675621,0.791130,0.797861,0.563368,0.843737,...,29,13,19,5,0.066667,0.062500,0.034483,0.076923,0.052632,0.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,Logistic_ClassWeights,cc_fraud_4,0.251029,0.975220,0.942276,0.564417,0.778441,0.839186,0.403509,0.938776,...,42,7,42,39,0.034483,0.023810,0.023810,0.142857,0.023810,0.025641
259,ROS_Logistic,cc_fraud_4,0.395190,0.979505,0.948780,0.588235,0.792116,0.834981,0.432692,0.918367,...,40,11,40,40,0.035714,0.024390,0.025000,0.090909,0.025000,0.025000
402,SMOTETomek_Logistic,cc_fraud_4,1.316931,0.950094,0.960976,0.647059,0.806696,0.833095,0.505747,0.897959,...,37,33,39,41,0.026316,0.027027,0.027027,0.030303,0.025641,0.024390
311,RUS_Logistic,cc_fraud_4,0.124177,0.958354,0.941463,0.555556,0.766180,0.823492,0.398230,0.918367,...,43,12,43,42,0.027778,0.023256,0.023256,0.083333,0.023256,0.023810


In [169]:
baseline_models = baseline_models.sort_values(by=['Dataset', 'Test_Overall_Score', 
                                                  'Test_Overall_Score_R'], ascending=[True, False, False]).copy()

# Now we'll use rank() to assign a row number within each 'Dataset'
baseline_models['RANK'] = baseline_models.groupby('Dataset').cumcount() + 1

best_baseline = baseline_models[baseline_models.RANK==1][['Model', 'Dataset','OverFitting_Flag',
                                                          'Easy_Nature','Test_Overall_Score',
                                                          'Test_F1 Score']].copy().rename({
    "Model":"BestBaseline",
    "Test_Overall_Score":"BaselineScore",
    "Test_F1 Score" : "BaselineF1"
},axis=1)

In [170]:
best_baseline

,BestBaseline,Dataset,OverFitting_Flag,Easy_Nature,BaselineScore,BaselineF1
533,XGB,Adult,0,0,0.799238,0.672906
534,XGB,Churn_Ecom,0,0,0.489681,0.186840
535,XGB,CoverType,0,0,0.841062,0.679181
536,XGB,GMC_Credit_Scoring,0,0,0.594550,0.287751
537,XGB,Nursery,0,1,0.913683,0.819672
538,XGB,PTB_Online,0,0,0.779816,0.625000
539,XGB,Taiwan_Credit_Scoring,0,0,0.637305,0.469307
540,XGB,Wine,0,0,0.707095,0.522613
541,XGB,cc_fraud_05,0,1,0.924240,0.866667
542,XGB,cc_fraud_1,0,1,0.915425,0.847458


In [171]:
best_model = s2[s2.rank_overall==1][['Model', 'Dataset','Test_Overall_Score','Test_F1 Score']].copy().rename({
                "Model":"BestModel",
                "Test_Overall_Score":"BestScore",
                'Test_F1 Score':"BestModelF1"
            },axis=1)

In [172]:
comp = best_baseline.merge(best_model, on="Dataset")

In [173]:
comp["PercentIncrease"] = ((comp.BestScore/comp.BaselineScore)-1)*100

In [175]:
comp["PercentIncreaseF1"] = ((comp.BestModelF1/comp.BaselineF1)-1)*100

In [177]:
comp.to_excel(join(RESULTS_PATH,"BEST_MODELS.xlsx"),index=False)

In [176]:
comp

,BestBaseline,Dataset,OverFitting_Flag,Easy_Nature,BaselineScore,BaselineF1,BestModel,BestScore,BestModelF1,PercentIncrease,PercentIncreaseF1
0,XGB,Adult,0,0,0.799238,0.672906,DCBoost,0.811382,0.718959,1.519449,6.843893
1,XGB,Churn_Ecom,0,0,0.489681,0.186840,DCBoost,0.551493,0.334053,12.622926,78.791123
2,XGB,CoverType,0,0,0.841062,0.679181,DCBoost,0.885990,0.774457,5.341736,14.028021
3,XGB,GMC_Credit_Scoring,0,0,0.594550,0.287751,DCBoost,0.620413,0.392131,4.350021,36.274075
4,XGB,Nursery,0,1,0.913683,0.819672,SSM,0.986653,0.972222,7.986352,18.611111
5,XGB,PTB_Online,0,0,0.779816,0.625000,ROS_XGB,0.799452,0.669246,2.517942,7.079304
6,XGB,Taiwan_Credit_Scoring,0,0,0.637305,0.469307,RUS_XGB,0.662472,0.530256,3.948973,12.986989
7,XGB,Wine,0,0,0.707095,0.522613,ROS_XGB,0.737603,0.589888,4.314523,12.872731
8,XGB,cc_fraud_05,0,1,0.924240,0.866667,SelfBoostingXGB,0.924473,0.866667,0.025155,0.000000
9,XGB,cc_fraud_1,0,1,0.915425,0.847458,IF_XGB,0.915461,0.847458,0.003913,0.000000


In [186]:
comp.columns

Index(['BestBaseline', 'Dataset', 'OverFitting_Flag', 'Easy_Nature',
       'BaselineScore', 'BaselineF1', 'BestModel', 'BestScore', 'BestModelF1',
       'PercentIncrease', 'PercentIncreaseF1'],
      dtype='object')

## Comparison of BEST BASELINE AND ALL OTHERS

In [178]:
other_models = s2[~s2.Model.isin(['XGB','Logistic','RandomForest'])][['Model', 'Dataset',
                                                                     'Test_Overall_Score',
                                                                     'Test_F1 Score']].copy().rename({
                "Test_Overall_Score":"ModelScore",
                'Test_F1 Score':"ModelF1"
            },axis=1)

In [181]:
comp2 = best_baseline.merge(other_models, on="Dataset")
comp2["PercentIncrease"] = ((comp2.ModelScore/comp2.BaselineScore)-1)*100

comp2["PercentIncreaseF1"] = ((comp2.ModelF1/comp2.BaselineF1)-1)*100

In [182]:
comp2

,BestBaseline,Dataset,OverFitting_Flag,Easy_Nature,BaselineScore,BaselineF1,Model,ModelScore,ModelF1,PercentIncrease,PercentIncreaseF1
0,XGB,Adult,0,0,0.799238,0.672906,ROS_XGB,0.808431,0.698178,1.150189,3.755599
1,XGB,Adult,0,0,0.799238,0.672906,XGB_ClassWeights,0.808483,0.697579,1.156719,3.666508
2,XGB,Adult,0,0,0.799238,0.672906,RUS_XGB,0.808393,0.696386,1.145422,3.489213
3,XGB,Adult,0,0,0.799238,0.672906,RUSBoost,0.801406,0.688927,0.271229,2.380856
4,XGB,Adult,0,0,0.799238,0.672906,EasyEnsemble,0.791130,0.675621,-1.014462,0.403345
...,...,...,...,...,...,...,...,...,...,...,...
515,RandomForest,cc_fraud_4,0,1,0.963742,0.934783,Logistic_ClassWeights,0.778441,0.564417,-19.227206,-39.620488
516,RandomForest,cc_fraud_4,0,1,0.963742,0.934783,ROS_Logistic,0.792116,0.588235,-17.808316,-37.072503
517,RandomForest,cc_fraud_4,0,1,0.963742,0.934783,SMOTETomek_Logistic,0.806696,0.647059,-16.295402,-30.779754
518,RandomForest,cc_fraud_4,0,1,0.963742,0.934783,RUS_Logistic,0.766180,0.555556,-20.499447,-40.568475


In [183]:
comp2.to_excel(join(RESULTS_PATH,"BEST_BASELINE_VS_EACH_MODEL.xlsx"),index=False)

In [184]:
comp2[comp2.Easy_Nature==0].groupby("Model").PercentIncrease.mean().sort_values(ascending=False)

Model
DCBoost                      4.764867
ROS_XGB                      2.561643
XGB_ClassWeights             2.309257
RUS_XGB                      1.418649
Tomek_XGB                    0.658625
PSM                          0.573347
SMOTETomek_XGB               0.460399
RUSBoost                     0.344363
SSM                          0.155190
SelfBoostingXGB              0.149998
SMOTEBoost                   0.089505
SMOTE_XGB                    0.026524
IF_XGB                      -0.148247
LOF_IF_XGB                  -0.281819
RAMOBoost                   -0.312172
LOF_XGB                     -0.487424
ADASYN_XGB                  -0.533164
EasyEnsemble                -0.953231
ROS_RandomForest            -1.597840
RandomFores_ClassWeights    -1.621961
RUS_RandomForest            -1.679868
SMOTETomek_RandomForest     -2.903327
SMOTE_RandomForest          -3.145136
SMOTEBagging                -3.372545
ADASYN_RandomForest         -3.895992
BalancedBagging             -4.020887
ROS_Lo

In [185]:
comp2[comp2.Easy_Nature==0].groupby("Model").PercentIncreaseF1.mean().sort_values(ascending=False)

Model
DCBoost                     24.223224
ROS_XGB                     15.509990
XGB_ClassWeights            15.058703
RUS_XGB                     13.056642
SMOTETomek_XGB              12.794273
RUSBoost                    11.624972
SMOTE_XGB                   11.462500
ADASYN_XGB                  10.198644
EasyEnsemble                 8.330936
RUS_RandomForest             7.986189
ROS_RandomForest             7.869553
RandomFores_ClassWeights     7.834403
SMOTEBoost                   6.479556
SMOTETomek_RandomForest      4.648169
RAMOBoost                    4.637952
SMOTE_RandomForest           4.142206
SMOTEBagging                 3.892735
Tomek_XGB                    3.242318
ADASYN_RandomForest          2.534248
BalancedBagging              2.445932
ROS_Logistic                 1.210645
SSM                          0.574949
PSM                          0.419127
Logistic_ClassWeights        0.247814
SelfBoostingXGB              0.229646
IF_XGB                      -0.364498
LOF_IF